In [1]:
import re
from collections import defaultdict
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

rules, updates = data.strip().split("\n\n")
rules = rules.split("\n")
updates = updates.split("\n")

In [3]:
## Part 1
# An update is accepted if none of the page-ordering rules is broken
# Sum the middle page number of each accepted update

# Opt 1: Turn every rule A|B into a regex for B with positive lookahead of A
#  -> if any of the rules match, reject the update 
# Opt 2 (probably more sensible): dict of rules {B: A}, keep a set of seen pages
#  For each page, if a rule applies, check that A hasn't been seen yet

def get_middle_number(xs: list):
    l = len(xs)
    return xs[l // 2]

In [4]:
# Opt 1, for fun:
rejection_patterns = []
for rule in rules:
    a, b = rule.split("|")
    # When we see B, do we see A ahead? If yes, reject.
    pattern = re.compile(rf"{b}(?=.*{a})")
    rejection_patterns.append(pattern)

# Note: this makes a pass through update for every rule, which scales extremely poorly!
accepted_updates = [update for update in tqdm(updates) if all((p.search(update) is None) for p in rejection_patterns)]
p1_result = sum(int(get_middle_number(update.split(","))) for update in accepted_updates)
p1_result

100%|██████████| 6/6 [00:00<00:00, 21399.51it/s]


143

In [5]:
## Part 2
# Keep only the rejected updates
# Reorder each of them so they break no rules
# Sum the middle page numbers again
rejected_updates = [update for update in updates if update not in accepted_updates]
len(rejected_updates)

3

In [6]:
# Sth like insertion sort?
# Parse rules to get the list of all Bs this A should be in front of {A: [B]}
# Keep a sorted sublist, and add new pages A into the largest index that's before all Bs (satisfies all rules)
rule_dict = defaultdict(set)
# Parse rules
for rule in rules:
    a, b = rule.split("|")
    rule_dict[a].add(b)

In [7]:
def reorder_update(pages: list, rule_dict=rule_dict):
    result = []
    for page in pages:
        # Get the index that we should insert before
        # Default to the end of the list
        i_candidates = [len(result)]
        for b in rule_dict[page]:
            try:
                i_candidates.append(result.index(b))
            except ValueError:
                pass
        result.insert(min(i_candidates), page)

    return result

In [8]:
p2_result = 0
for update_str in tqdm(rejected_updates):
    update = update_str.split(",")
    # print(f"Before: {update}")
    reordered_update = reorder_update(update)
    # print(f"After: {reordered_update}")
    p2_result += int(get_middle_number(reordered_update))

p2_result

100%|██████████| 3/3 [00:00<00:00, 7186.13it/s]


123